### 신경망 모델 구성하기
- FashionMNIST 데이터셋의 이미지들을 분류하는 신경망 구성

In [1]:
import os
import torch
from torch import nn     # 신경망을 구성하는데 필요한 모든 구성 요소들 제공
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### 학습을 위한 장치 얻기

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cpu device


### 클래스 정의하기

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device = device)
logits = model(X)                           # model.forward() : (x) / logits.size() = (1, 10)
pred_probab = nn.Softmax(dim = 1)(logits)   # [0, 1]로 scaling (sum to 1)
y_pred = pred_probab.argmax(dim = 1)
print(f'Predicted class: {y_pred}')

Predicted class: tensor([5])


### 모델 계층(Layer)

In [6]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


1. nn.Flatten

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


2. nn.Linear

In [8]:
layer1 = nn.Linear(in_features = 28 * 28, out_features = 20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


3. nn.ReLU

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.7028,  0.1240, -0.1343, -0.3060,  0.0752,  0.2318,  0.4740, -0.1087,
         -0.0405,  0.0374,  0.5884,  0.0795,  0.2573,  0.1576, -0.2356,  0.1981,
         -0.0522,  0.0670,  0.5634, -0.1215],
        [-0.7102,  0.1744,  0.1251, -0.0910, -0.3544,  0.2280,  0.5323,  0.1253,
         -0.3093,  0.0626,  0.4260,  0.1255,  0.6618,  0.3385,  0.0418,  0.1407,
         -0.1403,  0.0120,  0.4167, -0.1188],
        [-0.8503,  0.1020,  0.2596, -0.2336, -0.1066,  0.1367,  0.7367, -0.1176,
         -0.4444,  0.0555,  0.1399,  0.3513,  0.3577,  0.1693,  0.0856,  0.5790,
         -0.2805,  0.1058,  0.3868, -0.1471]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.0000, 0.1240, 0.0000, 0.0000, 0.0752, 0.2318, 0.4740, 0.0000, 0.0000,
         0.0374, 0.5884, 0.0795, 0.2573, 0.1576, 0.0000, 0.1981, 0.0000, 0.0670,
         0.5634, 0.0000],
        [0.0000, 0.1744, 0.1251, 0.0000, 0.0000, 0.2280, 0.5323, 0.1253, 0.0000,
         0.0626, 0.4260, 0.1255, 0.6618, 0.3385, 0.041

4. nn.Sequential

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)    # logits.size() : torch.Size([3, 10])
print(logits)

tensor([[-0.0857,  0.0795, -0.2202,  0.1652,  0.0990, -0.1084, -0.1047,  0.1225,
          0.0221,  0.1386],
        [-0.0704, -0.0062, -0.4138,  0.0372,  0.1390, -0.0691, -0.0496,  0.0769,
          0.0735, -0.0647],
        [-0.0103, -0.0388, -0.3705,  0.0347,  0.0376,  0.0040, -0.0058,  0.0383,
         -0.0006,  0.0703]], grad_fn=<AddmmBackward>)


5. nn.Softmax
- 각 class에 대한 예측 확률을 나타내도록 [0, 1] 범위로 scaling

In [11]:
softmax = nn.Softmax(dim = 1)    # dim : 값의 합이 1이 되는 차원
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.0901, 0.1063, 0.0788, 0.1158, 0.1084, 0.0881, 0.0884, 0.1110, 0.1004,
         0.1128],
        [0.0956, 0.1019, 0.0678, 0.1064, 0.1178, 0.0957, 0.0976, 0.1107, 0.1104,
         0.0961],
        [0.1007, 0.0979, 0.0703, 0.1054, 0.1057, 0.1022, 0.1012, 0.1058, 0.1017,
         0.1092]], grad_fn=<SoftmaxBackward>)


### 모델 매개변수
- 모델의 ```parameters()``` or ```named_parameters()```를 이용해 모든 매개변수에 접근 가능

In [12]:
print("Model Structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model Structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0266,  0.0242,  0.0219,  ..., -0.0148, -0.0325,  0.0253],
        [-0.0011,  0.0199, -0.0341,  ...,  0.0187, -0.0049, -0.0345]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([0.0347, 0.0192], grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0104, -0.0387, -0.0014,  ...,  0.0370, -0.0243,  0.0066],
        [-0.0294, -0.0404, -0.0268,  ..., -0.0131,  0.0348,  0.0283]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.bias | Size: 